<a href="https://colab.research.google.com/github/hellwalker3/so-vits-svc-LoveLive-notebook/blob/main/so-vits-svc-LoveLive-notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#0. So-vits-svcで必要なモジュールをインポート 
#(svc-develop-team/so-vits-svcと同じです。)
!pip install --upgrade pip setuptools numpy numba
!pip install pyworld praat-parselmouth fairseq tensorboardX

In [ ]:
#1. huggingface上のプログラムをダウンロード
!git lfs install
!git clone https://huggingface.co/spaces/YazawaSunrise/so-vits-svc-LoveLive

In [ ]:
#2. hubertのモデルをダウンロード
!wget https://github.com/bshall/hubert/releases/download/v0.1/hubert-soft-0d54a1f4.pt -O /content/so-vits-svc-LoveLive/hubert/hubert-soft-0d54a1f4.pt

In [ ]:
#3. プログラムを修正(引数を渡せるようにする。不要なプログラムインポート回避)

target_file = "/content/so-vits-svc-LoveLive/inference_main.py"
inference_main ="""
import argparse
import io
import logging
import time
from pathlib import Path

import librosa
import numpy as np
import soundfile

from inference import infer_tool
from inference import slicer
from inference.infer_tool import Svc

logging.getLogger('numba').setLevel(logging.WARNING)
chunks_dict = infer_tool.read_temp("inference/chunks_temp.json")

def main(model_path, config_path, clean_names, spk_list, trans, slice_db, wav_format):
    svc_model = Svc(model_path, config_path)
    infer_tool.mkdir(["raw", "results"])

    infer_tool.fill_a_to_b(trans, clean_names)
    for clean_name, tran in zip(clean_names, trans):
        raw_audio_path = f"raw/{clean_name}"
        if "." not in raw_audio_path:
            raw_audio_path += ".wav"
        infer_tool.format_wav(raw_audio_path)
        wav_path = Path(raw_audio_path).with_suffix('.wav')
        chunks = slicer.cut(wav_path, db_thresh=slice_db)
        audio_data, audio_sr = slicer.chunks2audio(wav_path, chunks)

        for spk in spk_list:
            audio = []
            for (slice_tag, data) in audio_data:
                print(f'#=====segment start, {round(len(data) / audio_sr, 3)}s======')
                length = int(np.ceil(len(data) / audio_sr * svc_model.target_sample))
                raw_path = io.BytesIO()
                soundfile.write(raw_path, data, audio_sr, format="wav")
                raw_path.seek(0)
                if slice_tag:
                    print('jump empty segment')
                    _audio = np.zeros(length)
                else:
                    out_audio, out_sr = svc_model.infer(spk, tran, raw_path)
                    _audio = out_audio.cpu().numpy()
                audio.extend(list(_audio))

            res_path = f'./results/{clean_name}_{tran}key_{spk}.{wav_format}'
            soundfile.write(res_path, audio, svc_model.target_sample, format=wav_format)

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Inference')
    parser.add_argument('-m', '--model_path', type=str, required=True, help='path to the trained model')
    parser.add_argument('-c', '--config_path', type=str, required=True, help='path to the configuration file')
    parser.add_argument('-n', '--clean_names', nargs='+', type=str, required=True, help='list of clean audio file names')
    parser.add_argument('-s', '--spk_list', nargs='+', type=str, required=True, help='list of speaker names')
    parser.add_argument('-t', '--trans', nargs='+', type=int, required=True, help='list of pitch shift values')
    parser.add_argument('-sd', '--slice_db', type=int, default=-40, help='threshold for silence removal')
    parser.add_argument('-wf', '--wav_format', type=str, default='flac', help='audio output format')

    
    args = parser.parse_args()
    
    args.clean_names = [args.clean_names] if isinstance(args.clean_names, str) else args.clean_names
    args.spk_list = [args.spk_list] if isinstance(args.spk_list, str) else args.spk_list
    args.trans = [args.trans] if isinstance(args.trans, int) else args.trans
  
    main(args.model_path, args.config_path, args.clean_names, args.spk_list, args.trans, args.slice_db, args.wav_format)
"""
with open(target_file, 'w') as f:
    f.write(inference_main)


target_file = '/content/so-vits-svc-LoveLive/inference/infer_tool.py'
with open(target_file, 'r') as f:
    contents = f.read()

# 置換対象の文字列と置換後の文字列を指定する
old_str = 'import maad'
new_str = '#import maad'
# 文字列置換を実行する
contents = contents.replace(old_str, new_str)
# 上書き保存する
with open(target_file, 'w') as f:
    f.write(contents)

In [ ]:
#4. ディレクトリ移動
%cd /content/so-vits-svc-LoveLive

In [ ]:
#5. /content/so-vits-svc-LoveLive/rawフォルダに声のみのwavファイルを配置します。 (例はsample.wav)
from IPython.display import Audio, display
display(Audio(f"/content/so-vits-svc-LoveLive/raw/sample.wav"))

In [ ]:
#6. コマンド実行 
#(音楽ファイルのサイズは小さくしないとcuda out of memoryになります。20sは大丈夫でした。長いファイルは適当に分割して統合すると良さそうです。)

wav_filename =  "sample" #/content/so-vits-svc-LoveLive/rawフォルダ内のwavファイルのファイル名
group_name = "Niji2" #/content/so-vits-svc-LoveLive/checkpoints内のフォルダ名
target_model_file = f"/content/so-vits-svc-LoveLive/checkpoints/{group_name}/model.pth" #学習済みモデルのパス
config_path = f"/content/so-vits-svc-LoveLive/checkpoints/{group_name}/config.json" #configのパス
speaker =  "Setsuna" #configのspeaker
wav_format  = "wav" #出力音楽ファイルの拡張子

!python inference_main.py -n {wav_filename} -m {target_model_file} -c {config_path} -s {speaker} -t 0 -sd -40 -wf {wav_format} 
# /content/so-vits-svc-LoveLive/resultsフォルダに出力されます。

In [ ]:
#7. 再生
display(Audio(f"/content/so-vits-svc-LoveLive/results/{wav_filename}_0key_{speaker}.{wav_format}", autoplay=True))